In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import f1_score

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
class2idx = {
    'no_relevant': 0,
    'relevant_minus': 1,
    'relevant': 2,
    'relevant_plus': 3,
}

idx2class = {v: k for k, v in class2idx.items()}

In [4]:
# posts = pd.read_parquet("/kaggle/input/russian-ai-olympiad-2024-b/items.parquet")

In [5]:
# itemId2idx = dict()
# for i, row in tqdm(posts.iterrows(), total=len(posts)):
#     itemId2idx[row['itemId']] = i

In [6]:
# train = pd.read_csv("/kaggle/input/russian-ai-olympiad-2024-b/train.csv")
# test = pd.read_csv("/kaggle/input/russian-ai-olympiad-2024-b/test.csv")
# subm = pd.read_csv("/kaggle/input/russian-ai-olympiad-2024-b/sample_submission.csv")

In [7]:
# train = train[~train['target'].isin(['doubles', 'unavailable'])]

In [8]:
# tqdm.pandas()

# def get_data_from_posts(row):
#     idx1, idx2 = itemId2idx[row['leftItemId']], itemId2idx[row['rightItemId']]
#     post1, post2 = posts.iloc[idx1], posts.iloc[idx2]

#     if row.get('target') is not None:
#         row['target'] = class2idx[row['target']]
    
#     row['text1'] = post1['title']
    
#     row['text2'] = post2['title']

#     row['same_author'] = 1 if post1['authorId'] == post2['authorId'] else 0
    
#     return row
    

# train = train.progress_apply(get_data_from_posts, axis=1)
# train.drop(columns=['leftItemId', 'rightItemId'], inplace=True)

# test = test.progress_apply(get_data_from_posts, axis=1)
# test.drop(columns=['leftItemId', 'rightItemId'], inplace=True)

In [9]:
# train.to_parquet("new_train.parquet")
# test.to_parquet("new_test.parquet")

In [10]:
train = pd.read_parquet("/kaggle/input/russian-ai-olympiad-2024-b/new_train.parquet")
test = pd.read_parquet("/kaggle/input/russian-ai-olympiad-2024-b/new_test.parquet")
subm = pd.read_csv("/kaggle/input/russian-ai-olympiad-2024-b/sample_submission.csv")

train['text'] = train['text1'] + train['text2']
test['text'] = test['text1'] + test['text2']

In [11]:
from sklearn.model_selection import train_test_split

# train, _ = train_test_split(train, test_size=0.6, stratify=train['target'], random_state=42)

train, valid = train_test_split(train, test_size=0.05, stratify=train['target'], random_state=42)

train.reset_index(drop=True, inplace=True)
valid.reset_index(drop=True, inplace=True)

In [12]:
!pip install --upgrade protobuf==3.20.3 -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 6.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 3.20.3 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
tensorflow-metadata 1.17.2 requires protobuf>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
pydrive2 1.21.3 requires cryptography<44, bu

In [13]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_checkpoint = "DeepPavlov/rubert-base-cased-sentence"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=4).to(device)

tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

2025-11-13 06:28:38.188598: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763015318.365443      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763015318.410215      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


pytorch_model.bin:   0%|          | 0.00/711M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased-sentence and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
%%time

sentences = ["Hello World", "Привет Мир"]

encoded_input = tokenizer(sentences, padding=True, truncation=True, max_length=64, return_tensors='pt')
encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
with torch.no_grad():
    logits = model(**encoded_input).logits

probs = torch.softmax(logits, dim=-1)
preds = torch.argmax(probs, dim=-1)
preds

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

CPU times: user 112 ms, sys: 60 ms, total: 172 ms
Wall time: 307 ms


tensor([3, 0], device='cuda:0')

In [15]:
model.device

device(type='cuda', index=0)

In [16]:
class_counts = torch.tensor(train['target'].value_counts().sort_index().values, dtype=torch.float)
weights = 1.0 / class_counts  

weights = weights / weights.sum() * len(class_counts)  
weights

tensor([0.9734, 0.5328, 0.5495, 1.9443])

In [24]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import DataCollatorWithPadding

BATCH_SIZE = 64

data_collator = DataCollatorWithPadding(tokenizer)

class PostDataset(Dataset):
    def __init__(self, text, target=None):
        self.text = text
        self.encodings = tokenizer(self.text, padding=False, truncation=True, max_length=128, return_tensors=None)
        self.target = target
    
    def __len__(self):
        return len(self.text)
    def __getitem__(self, idx):
        enc = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        if self.target is not None:
            enc['labels'] = torch.tensor(self.target[idx], dtype=torch.long)
        return enc

train_ds = PostDataset(train['text'].tolist(), train['target'].tolist())
valid_ds = PostDataset(valid['text'].tolist(), valid['target'].tolist())

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, collate_fn=data_collator)
valid_loader = DataLoader(valid_ds, batch_size=BATCH_SIZE, collate_fn=data_collator)

In [25]:
epochs = 1
log_rate = 1
batch_log_rate = 50

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=4).to(device)
model.to(device)

loss_fn = nn.CrossEntropyLoss(weight=weights.to(device))
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5, weight_decay=1e-4)
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs*len(train_loader))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased-sentence and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
for epoch in tqdm(range(epochs), desc='Training'):
    model.train()
    running_train_loss = 0
    all_preds = []
    i = 0
    for X in (pbar := tqdm(train_loader, leave=False, desc='Train DataLoader')):
        i += 1
        y = X['labels'].to(device)
        X = {k: v.to(device) for k, v in X.items() if k != 'labels'}
        logits = model(**X).logits
        loss = loss_fn(logits, y)

        probs = torch.argmax(torch.softmax(logits, dim=-1), dim=-1)
        all_preds.extend(probs.detach().cpu().tolist())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # scheduler.step()

        running_train_loss += loss.item()
        if i%batch_log_rate==0 or i==len(train_loader):
            print(f"Batch {i}/{len(train_loader)} | Train Loss: {running_train_loss/i:.5f}")
            # print(f"Diversity: {torch.bincount(torch.tensor(all_preds)).tolist()}")
        pbar.set_postfix({'loss': f"{running_train_loss/i:.5f}",}) 
                          # "diversity": torch.bincount(torch.tensor(all_preds)).tolist()})
        
    if (epoch+1)%log_rate==0:
        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {running_train_loss/i:.5f}")

    model.eval()
    all_preds, all_targets = [], []
    i = 0
    for X in (pbar := tqdm(valid_loader, leave=False, desc='Valid DataLoader')):
        i += 1
        y = X['labels'].to(device)
        X = {k: v.to(device) for k, v in X.items() if k != 'labels'}
        with torch.no_grad():
            logits = model(**X).logits
        probs = torch.argmax(torch.softmax(logits, dim=-1), dim=-1)
        all_preds.extend(probs.detach().cpu().tolist())
        all_targets.extend(y.cpu().tolist())
    
    f1 = f1_score(all_targets, all_preds, average='weighted')
    if (epoch+1)%log_rate==0:
        print(f"Weighted F1 Score: {f1:.5f}")
        print(f"Diversity: {torch.bincount(torch.tensor(all_preds)).tolist()}")

Training:   0%|          | 0/1 [00:00<?, ?it/s]

Train DataLoader:   0%|          | 0/3095 [00:00<?, ?it/s]

Batch 50/3095 | Train Loss: 1.06428
Batch 100/3095 | Train Loss: 1.05440
Batch 150/3095 | Train Loss: 1.04107
Batch 200/3095 | Train Loss: 1.03342
Batch 250/3095 | Train Loss: 1.02329
Batch 300/3095 | Train Loss: 1.01627
Batch 350/3095 | Train Loss: 1.00681
Batch 400/3095 | Train Loss: 1.00745
Batch 450/3095 | Train Loss: 1.00653
Batch 500/3095 | Train Loss: 1.00247
Batch 550/3095 | Train Loss: 1.00082
Batch 600/3095 | Train Loss: 1.00022
Batch 650/3095 | Train Loss: 0.99847
Batch 700/3095 | Train Loss: 0.99499
Batch 750/3095 | Train Loss: 0.99383
Batch 800/3095 | Train Loss: 0.99374
Batch 850/3095 | Train Loss: 0.99314
Batch 900/3095 | Train Loss: 0.99254
Batch 950/3095 | Train Loss: 0.99212
Batch 1000/3095 | Train Loss: 0.99007
Batch 1050/3095 | Train Loss: 0.98900
Batch 1100/3095 | Train Loss: 0.98740
Batch 1150/3095 | Train Loss: 0.98440
Batch 1200/3095 | Train Loss: 0.98270
Batch 1250/3095 | Train Loss: 0.98085
Batch 1300/3095 | Train Loss: 0.98088
Batch 1350/3095 | Train Loss: 0.

Valid DataLoader:   0%|          | 0/163 [00:00<?, ?it/s]

Weighted F1 Score: 0.51900
Diversity: [1856, 3856, 3345, 1368]


In [33]:
test_ds = PostDataset(test['text'].tolist())

test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, collate_fn=data_collator)

In [34]:
model.eval()
all_preds = []
i = 0
for X in (pbar := tqdm(test_loader, desc='Test DataLoader')):
    i += 1
    X = {k: v.to(device) for k, v in X.items()}
    with torch.no_grad():
        logits = model(**X).logits
    probs = torch.argmax(torch.softmax(logits, dim=-1), dim=-1)
    all_preds.extend(probs.detach().cpu().tolist())

Test DataLoader:   0%|          | 0/807 [00:00<?, ?it/s]

In [35]:
subm['target'] = list(map(idx2class.get, all_preds))
subm.to_csv("submission.csv", index=False)
subm

,Unnamed: 0,target
0,0,relevant
1,1,relevant
2,2,relevant_plus
3,3,no_relevant
4,4,relevant_minus
...,...,...
51631,51631,relevant_minus
51632,51632,relevant
51633,51633,relevant
51634,51634,relevant_minus


In [36]:
subm['target'].value_counts()

target
relevant_minus    19334
relevant          16579
no_relevant        8842
relevant_plus      6881
Name: count, dtype: int64